In [1]:
#Created 2023-01-09 @ 1900 Hrs
#Final Product at 2023-01-12 @ 1020 Hrs

###packages used###

import requests  #pip install requests
from bs4 import BeautifulSoup #pip install bs4
import re
import pandas as pd #pip install pandas
import openpyxl #pip install openpyxl

import xlwings as xw #pip install xlwings

from selenium import webdriver #pip install selenium
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager #pip install web-driver
from selenium.webdriver.common.by import By


In [8]:
start_url='https://www.sdn.sg/forindividuals/Pages/Dating-Events-Month.aspx' 
# the url used for reference >> one indicating dating events hosted in the current year
start_sheet=requests.get(start_url) 

In [75]:
driver=webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()

C:\Users\acer\AppData\Local\Temp\ipykernel_8108\2317457023.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


In [74]:
def get_data(content): # function specific to the url above; used to extract certain information form the html content of the url
    soup=BeautifulSoup(content, 'html.parser')
    info=soup.find_all(class_="event_cal_height1") # information can be found under this class in the url

    df=pd.DataFrame(columns=['Date','Event','Agency','Cost','Time','Venue','Details']) # creates empty dataframe

    for i in info:
        url=re.findall('<a href="(\S*)">',str(i.contents)) # uses regex to pull out relevant info
        details=('https://www.sdn.sg'+url[0]) # full url for further info on event (useful due to unique event id it possesses)
        extract=[x for x in re.findall('\S*',str(i.text)) if x!='']
        date=(' '.join(extract[:2])) # pulls out the date (in format of mon, day)
        name=(' '.join(extract[2:extract.index('Agency')-1])) # pulls out name of event
        agency=(' '.join(extract[extract.index('Agency')+2:extract.index('Cost')])) # pulls out agency hosting the event
        cost=float(' '.join(extract[extract.index('Cost')+3:extract.index('Time')])) # pulls out the cost of the event (digits only)
        time=(' '.join(extract[extract.index('Time')+2:extract.index('Venue')])) # pulls time of the event (from what time to what time)
        venue=(' '.join(extract[extract.index('Venue')+2:])) # pulls out location of event
        df.loc[len(df)]=[date,name,agency,cost,time,venue,details] # stores all that info in the df

        df=df.iloc[::-1] # reverses order of dataframe (so that recent entries are stored on top)
        df.index=range(0,len(df)) # ensures that index of df is continuous
    return df


In [76]:
driver.get(start_url)
curr_page_source=driver.page_source
og=pd.DataFrame(columns=['Date','Event','Agency','Cost','Time','Venue','Details'])

soup=BeautifulSoup(driver.page_source, 'html.parser')


In [77]:

parent=soup.find('div',{'class':'pageLink'}) # search for pagination
children= parent.findChildren('a',recursive=False)

upper=soup.find('div',{'class':'eventmonth'})# search for month

og
for e in upper.find_all(class_='button'):
    id=(re.findall('id="(.*)" style',str(e)))[0]
    driver.find_element(By.ID,id).click() # finds and clicks each page button on the link (necessary as url is hyperlinked)
    # diff 
    mon=pd.DataFrame(columns=['Date','Event','Agency','Cost','Time','Venue','Details'])
    if get_data(driver.page_source).empty:
        print('Done')
        break 
    else:
        for i,child in enumerate(children):
            driver.find_element(By.LINK_TEXT,(str(i+1))).click()
            curr_page_source=driver.page_source
            if (get_data(curr_page_source).empty):
                print('Empty')
                break 
            else:
                n=get_data(curr_page_source)
                mon=pd.concat([mon,n],axis=0)
        mon=mon.sort_values('Date',ascending=False) 
    og=pd.concat([mon,og],axis=0)


Empty
Done


In [78]:
og.index=(list(range(1,len(og)+1)))

In [79]:
excel_file=r"C:\Users\acer\OneDrive\Desktop\Results.xlsx" # Dependent on location and all; to create excel sheet beforehand on personal computer
xcel=pd.read_excel(excel_file)  

wb=openpyxl.load_workbook(excel_file,data_only=True)
ws=wb['Sheet1']
clr=openpyxl.styles.PatternFill(fill_type=None)

for row in ws["A1":"H2000"]:
    for cell in row:
        cell.fill = clr

wb.save(filename=excel_file)

rows=[]
for i,det in enumerate(og['Details']): # compares new entries based on details (owing to unique event id present in each url)
 if det not in list(xcel['Details']): # thus; even if event is added on a previous date; will still appear
    i+=2
    rows.append(str(i))

In [80]:
toxl=og
toxl.to_excel(excel_file)
wbx = xw.Book(excel_file)
for e in rows:
    wbx.sheets['Sheet1'].range(f'A{e}:H{e}').color = (255,234,0)
wbx.save(excel_file)
wbx.close()